# Disposition FM model

Import libraries and packages
Define the NN-model

Train said model:
Optimizer, learning rate.
Define loss function (MSE)

Implement a for-loop that goes for the desired amount of training steps
For each iteration:
Generate samples from a dataset (e.g. 256 trajectories from a 10 000 trajectory dataset)
Convert into PyTorch tensor

Generate intial state at random, make sure they have the same shape as the samples
Sample random time steps, assign each trajectory with a time sampled from a uniform distribution [0,1]

Interpolate between the dataset sample (x1) and the initial state (x0)
The interpolated state (xt) is then sued to compute the target velocity field
dxt = x1-x0
dxt is the velocity field, defined as the difference between x1 and x0 (i.e. the direction for the trajectory to move)

Optimize the flow model, measure the loss, store the loss
Backpropagation, update parameters

Sample from the trained model

Plot progression
Store plots

# Trajectory considerations:
For full trajectories instead of points, each trajectory should be stored as a sequence, where x_t represents the system's state at time t.
Instead of sampling x1 and x0 randomly, I should instead sample a full trajectory, choose two time steps within the traj and interpolate between the x(t_0) and x(t_1) for those time steps.
Interpolation: alpha=(t-t0)/(t1-t0)
Loss function should minimize diff between learned vel: flow(xt,t)  and true vel: dxt

In [ ]:
import torch
from torch import nn, Tensor
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class Flow(nn.Module):
    def __init__(self, dim: int = 2, h: int = 64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim + 1, h), nn.ELU(),
            nn.Linear(h, h), nn.ELU(),
            nn.Linear(h, h), nn.ELU(),
            nn.Linear(h, dim))

    def forward(self, t: Tensor, x_t: Tensor) -> Tensor:
        return self.net(torch.cat((t, x_t), -1))

    def step(self, x_t: Tensor, t_start: Tensor, t_end: Tensor) -> Tensor:
        t_start = t_start.view(1, 1).expand(x_t.shape[0], 1)
        return x_t + (t_end - t_start) * self(t=t_start + (t_end - t_start) / 2, x_t= x_t + self(x_t=x_t, t=t_start) * (t_end - t_start) / 2)

In [ ]:
flow = Flow()
optimizer = torch.optim.AdamW(flow.parameters(), 1e-3)
loss_fn = nn.MSELoss()

In [ ]:
# Training loop
dataset = imported_dataset ## Fiks

training_steps = 10000
for _ in range(training_steps):
    trajectory, times = dataset[random.randint(0, len(dataset)-1)]

    t0_idx, t1_idx = torch.randint(0, len(times)-1 , (2,)) # Take two random samples from a trajectory
    t0, t1 = times[t0_idx], times[t1_idx] # get the corresponding time to the two random samples
    x_t0, x_t1 = trajectory[t0_idx], trajectory[t1_idx] # get the corresponding trajectory "positions"

    alpha = torch.rand(1) # Interpolation factor
    t = (1 - alpha) * t0 + alpha * t1_idx # Interpolate time 
    x_t = (1 - alpha) * x_t0 + alpha * x_t1 # Interpolate "location"
    dx_t = (x_t1 - x_t0) / (t1 - t0) # Calculate velocity field

    optimizer.zero_grad() # Optimize
    loss_fn(flow(x_t, t), dx_t).backward() # Backpropagate
    optimizer.step() # Next step
